# Lab 3: 1D Finite Differences

## Thomas Algorithm 

### Code Deliverable 

In [15]:
#import external modules 
import numpy as np

def thomas_solver(a, b, c, d):
    
    """
    The Thomas Algorithm is a simplified form of Gaussian elimination that 
    can be used to solve tridiagonal systems of equations.
    
    Input:
        a, b, c, d - numpy vectors that make up tridiagonal matrix and solution d 
    Output: 
        x - solution numpy vector
    """
    
    #initialize number of equations
    #this will be the length of vector d 
    n = len(d)
    
    #make a copy of each array 
    #can use the map function to map an array function to each variable 
    aa, bb, cc, dd = (x.astype(float) for x in (a, b, c, d))
    
    #loop for n iterations 
    for i in range(1, n):
        
        #apply the Thomas Algorithm 
        m = aa[i - 1] / bb[i - 1]
        bb[i] = bb[i] - m * cc[i - 1]
        dd[i] = dd[i] - m * dd[i - 1]
        
    #assign solution x to main diagonal b 
    x = bb
    x[-1] = dd[-1] / bb[-1]
    
    for i in range(n - 2, -1, -1):
        x[i] = (dd[i] - cc[i] * x[i + 1]) / bb[i]
        
    #return solution
    return x 

### Thomas Algorithm Test

In [22]:
#initialize arrays a b c and d
a = np.full([9], -1)
b = np.full([10], 3)
c = np.full([9], -1)
d = np.array([2,1,1,1,1,1,1,1,1,2])

#call thomas_solver function with arrays 
result = thomas_solver(a, b, c, d)
print(result)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [37]:
A = np.array([[3, -1, 0, 0, 0, 0, 0, 0, 0, 0],
             [-1, 3, -1, 0, 0, 0, 0, 0, 0, 0],
             [0, -1, 3, -1, 0, 0, 0, 0, 0, 0],
             [0, 0, -1, 3, -1, 0, 0, 0, 0, 0],
             [0, 0, 0, -1, 3, -1, 0, 0, 0, 0],
             [0, 0, 0, 0, -1, 3, -1, 0, 0, 0],
             [0, 0, 0, 0, 0, -1, 3, -1, 0, 0],
             [0, 0, 0, 0, 0, 0, -1, 3, -1, 0],
             [0, 0, 0, 0, 0, 0, 0, -1, 3, -1],
             [0, 0, 0, 0, 0, 0, 0, 0, -1, 3]])

d = np.array([2,1,1,1,1,1,1,1,1,2])
print(np.linalg.solve(A,d))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


## Finite Differences

### Code Deliverable

In [ ]:
def finite_difference_sparce(f, a, b, ua, ub, N):
    
    